## Import all relevant packages

In [1]:
#Import all relevant packages
import pandas as pd
# import geopandas as gpd

import numpy as np
# from shapely.geometry import Point
# from tqdm import tqdm, tqdm_notebook

import missingno as msno
from pandas_profiling import ProfileReport

import seaborn as sns
# import folium
print('All packages imported')

All packages imported


In [19]:
pd.__version__

'1.3.5'

In [ ]:
# url = 'GitHub Link'
# df1 = pd.read_csv(url)
# df1.head()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')



## Load all historical data from Jan 2021 to Jan 2022

In [2]:
def load_data() -> pd.DataFrame:
  dfPandas = pd.DataFrame()
  for x in range(202101, 202113):
    url = 'https://github.com/HakimIndriawan/GDA_Certificate-CaseStudy1-BikeSharing/blob/main/Data-divvy-tripdata/' + str(x) + '-divvy-tripdata.csv?raw=true'
    print(x)
    dfTemp = pd.read_csv(url)
    dfPandas = pd.concat([dfPandas, dfTemp], ignore_index=True, sort=False)
  print('202201')
  dfPandas = pd.concat([dfPandas,
                          pd.read_csv('https://github.com/HakimIndriawan/GDA_Certificate-CaseStudy1-BikeSharing/blob/main/Data-divvy-tripdata/202201-divvy-tripdata.csv?raw=true')],
                        ignore_index=True, sort=False) 
  return dfPandas
dfTripData = load_data()
dfTripData.head()

202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111
202112
202201


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,E19E6F1B8D4C42ED,electric_bike,2021-01-23 16:14:19,2021-01-23 16:24:44,California Ave & Cortez St,17660,NaN,NaN,41.900341,-87.696743,41.89,-87.72,member
1,DC88F20C2C55F27F,electric_bike,2021-01-27 18:43:08,2021-01-27 18:47:12,California Ave & Cortez St,17660,NaN,NaN,41.900333,-87.696707,41.90,-87.69,member
2,EC45C94683FE3F27,electric_bike,2021-01-21 22:35:54,2021-01-21 22:37:14,California Ave & Cortez St,17660,NaN,NaN,41.900313,-87.696643,41.90,-87.70,member
3,4FA453A75AE377DB,electric_bike,2021-01-07 13:31:13,2021-01-07 13:42:55,California Ave & Cortez St,17660,NaN,NaN,41.900399,-87.696662,41.92,-87.69,member
4,BE5E8EB4E7263A0B,electric_bike,2021-01-23 02:24:02,2021-01-23 02:24:45,California Ave & Cortez St,17660,NaN,NaN,41.900326,-87.696697,41.90,-87.70,casual


In [3]:
# Calculate time spent on each ride
def calculate_timespent(dfPandas) -> pd.DataFrame:
  dfPandas['started_time'] = pd.to_datetime(dfPandas['started_at'].values)
  dfPandas["ended_time"] = pd.to_datetime(dfPandas["ended_at"].values)
  dfPandas["timespent"] = dfPandas["ended_time"].values - dfPandas["started_time"].values
  return dfPandas
dfTripData = calculate_timespent(dfTripData)
dfTripData.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,started_time,ended_time,timespent
0,E19E6F1B8D4C42ED,electric_bike,2021-01-23 16:14:19,2021-01-23 16:24:44,California Ave & Cortez St,17660,NaN,NaN,41.900341,-87.696743,41.89,-87.72,member,2021-01-23 16:14:19,2021-01-23 16:24:44,0 days 00:10:25
1,DC88F20C2C55F27F,electric_bike,2021-01-27 18:43:08,2021-01-27 18:47:12,California Ave & Cortez St,17660,NaN,NaN,41.900333,-87.696707,41.90,-87.69,member,2021-01-27 18:43:08,2021-01-27 18:47:12,0 days 00:04:04
2,EC45C94683FE3F27,electric_bike,2021-01-21 22:35:54,2021-01-21 22:37:14,California Ave & Cortez St,17660,NaN,NaN,41.900313,-87.696643,41.90,-87.70,member,2021-01-21 22:35:54,2021-01-21 22:37:14,0 days 00:01:20
3,4FA453A75AE377DB,electric_bike,2021-01-07 13:31:13,2021-01-07 13:42:55,California Ave & Cortez St,17660,NaN,NaN,41.900399,-87.696662,41.92,-87.69,member,2021-01-07 13:31:13,2021-01-07 13:42:55,0 days 00:11:42
4,BE5E8EB4E7263A0B,electric_bike,2021-01-23 02:24:02,2021-01-23 02:24:45,California Ave & Cortez St,17660,NaN,NaN,41.900326,-87.696697,41.90,-87.70,casual,2021-01-23 02:24:02,2021-01-23 02:24:45,0 days 00:00:43


In [5]:
def haversine(lat1, lng1, lat2, lng2):
  # km constant
  metric_constant = 6371
  lat1, lng1, lat2, lng2 = map(np.deg2rad, [lat1, lng1, lat2, lng2])
  dlat = lat2 - lat1
  dlng = lng2 - lng1
  a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlng/2)**2
  c = 2 * np.arcsin(np.sqrt(a))
  km = metric_constant * c
  return km
def calculate_distance(dfPandas) -> pd.DataFrame:
  dfPandas["distance"] = haversine(dfPandas["start_lat"].values, dfPandas["start_lng"].values, dfPandas["end_lat"].values, dfPandas["end_lng"].values)
  return dfPandas
dfTripData = calculate_distance(dfTripData)
dfTripData.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,started_time,ended_time,timespent,distance
0,E19E6F1B8D4C42ED,electric_bike,2021-01-23 16:14:19,2021-01-23 16:24:44,California Ave & Cortez St,17660,NaN,NaN,41.900341,-87.696743,41.89,-87.72,member,2021-01-23 16:14:19,2021-01-23 16:24:44,0 days 00:10:25,2.242244
1,DC88F20C2C55F27F,electric_bike,2021-01-27 18:43:08,2021-01-27 18:47:12,California Ave & Cortez St,17660,NaN,NaN,41.900333,-87.696707,41.90,-87.69,member,2021-01-27 18:43:08,2021-01-27 18:47:12,0 days 00:04:04,0.556327
2,EC45C94683FE3F27,electric_bike,2021-01-21 22:35:54,2021-01-21 22:37:14,California Ave & Cortez St,17660,NaN,NaN,41.900313,-87.696643,41.90,-87.70,member,2021-01-21 22:35:54,2021-01-21 22:37:14,0 days 00:01:20,0.280031
3,4FA453A75AE377DB,electric_bike,2021-01-07 13:31:13,2021-01-07 13:42:55,California Ave & Cortez St,17660,NaN,NaN,41.900399,-87.696662,41.92,-87.69,member,2021-01-07 13:31:13,2021-01-07 13:42:55,0 days 00:11:42,2.248210
4,BE5E8EB4E7263A0B,electric_bike,2021-01-23 02:24:02,2021-01-23 02:24:45,California Ave & Cortez St,17660,NaN,NaN,41.900326,-87.696697,41.90,-87.70,casual,2021-01-23 02:24:02,2021-01-23 02:24:45,0 days 00:00:43,0.275752


In [6]:
dfTripDataM = dfTripData[dfTripData.timespent >= np.timedelta64(1, 's')]
dfTripDataM.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,started_time,ended_time,timespent,distance
0,E19E6F1B8D4C42ED,electric_bike,2021-01-23 16:14:19,2021-01-23 16:24:44,California Ave & Cortez St,17660,NaN,NaN,41.900341,-87.696743,41.89,-87.72,member,2021-01-23 16:14:19,2021-01-23 16:24:44,0 days 00:10:25,2.242244
1,DC88F20C2C55F27F,electric_bike,2021-01-27 18:43:08,2021-01-27 18:47:12,California Ave & Cortez St,17660,NaN,NaN,41.900333,-87.696707,41.90,-87.69,member,2021-01-27 18:43:08,2021-01-27 18:47:12,0 days 00:04:04,0.556327
2,EC45C94683FE3F27,electric_bike,2021-01-21 22:35:54,2021-01-21 22:37:14,California Ave & Cortez St,17660,NaN,NaN,41.900313,-87.696643,41.90,-87.70,member,2021-01-21 22:35:54,2021-01-21 22:37:14,0 days 00:01:20,0.280031
3,4FA453A75AE377DB,electric_bike,2021-01-07 13:31:13,2021-01-07 13:42:55,California Ave & Cortez St,17660,NaN,NaN,41.900399,-87.696662,41.92,-87.69,member,2021-01-07 13:31:13,2021-01-07 13:42:55,0 days 00:11:42,2.248210
4,BE5E8EB4E7263A0B,electric_bike,2021-01-23 02:24:02,2021-01-23 02:24:45,California Ave & Cortez St,17660,NaN,NaN,41.900326,-87.696697,41.90,-87.70,casual,2021-01-23 02:24:02,2021-01-23 02:24:45,0 days 00:00:43,0.275752


In [13]:
# profile = ProfileReport(df1)
# profile.to_widgets()
# profile.to_file("pandas_profile.html")
# profile2 = ProfileReport(df1, explorative=True)
# profile2.to_widgets()
# msno.matrix(df1)
# msno.bar(df1)
df1['timespent_float'] = df1['timespent'].values.astype(int) / 1000
df1['timespent_float']

Process ForkPoolWorker-3:
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/usr/lib/python3.7/mu

KeyboardInterrupt: ignored

In [ ]:
filteredDf = df1[(df1.sditance <= 100) & (df1.timespent_float <= 2.5)]

# filteredDf = np.where((df1['distance']<=200))
# filteredDf.head()
sns.scatterplot(data=filteredDf, x="timespent_float", y="distance", hue="member_casual")


In [ ]:
# loc_center = [df1["start_lat"].mean(), df1["start_lng"].mean()]
# map1 = folium.Map(location = loc_center, tiles="OpenStreetMap", control_scale=True)
# # for index, loc in df1.iterrows():
# #     if loc['member_casual']=='member':
# #         color = 'blue'
# #     elif loc['member_casual']=='casual':
# #         color = "green"
# #     else:
# #         color = 'black'
# #     folium.RegularPolygonMarker([loc['start_lat'], loc['start_lng']], fill_color=color, number_of_sides=3, radius=6, rotation=30).add_to(map1)
# df1.apply(lambda row:folium.CircleMarker(location=[row["start_lat"], row["start_lng"]], 
#                                               radius=10, popup=row['ride_id'])
#                                              .add_to(map1), axis=1)
# # folium.LayerControl().add_to(map1)
# # map1